In [1]:
import numpy as np
import math
from skimage.transform import resize, rotate
import matplotlib.pyplot as plt

In [44]:
def append1s (X):
    b = np.ones((X.shape[0], 1))
    Xwithb = np.append(X, b, axis = 1)
    Xtilde = Xwithb
    return Xtilde

def shuffleData(X, y):
    shuffle = np.random.permutation(np.shape(X)[0])
    shuffled_X = X[shuffle]
    shuffled_y = y[shuffle]
    return shuffled_X, shuffled_y

def createBatches(X, y, ntilde):
    num_batches = math.floor(np.shape(X)[0] / ntilde)
    X_batches = np.split(X, num_batches)
    y_batches = np.split(y, num_batches)
    return np.array(X_batches), np.array(y_batches)
        
def softmax(X, w):
    z = np.dot(X, w)
    exp_z = np.exp(z)
    sum_exp_z = exp_z.sum(axis = 1)
    norm_exp_z = (exp_z.T / sum_exp_z).T
    return norm_exp_z

def oneHotEncode(y):
    onehot_y = np.zeros(np.shape(y))
    encoded_idx = np.argmax(y, axis = 1)
    idx = np.arange(np.shape(y)[0])
    onehot_y[idx, encoded_idx] = 1
    return onehot_y

def crossEntropy(y, yhat):
    return -1 / np.shape(y)[0] * np.sum(y * np.log(yhat), axis = 1)

def gradfCE(X, y, w):
    yhat = softmax(X, w)
    gradient = -1 / np.shape(X)[0] * np.dot(X.T, y - yhat)
    return gradient

def gradDescent(X, y, epochs = 500, ntilde = 100, alpha = .1):
    sigma = .1
    w = sigma * np.random.randn(np.shape(X)[1], 10)
    for epoch in range(epochs):
        X, y = shuffleData(X, y)
        X_batches, y_batches = createBatches(X, y, 100)
        printBool = True
        for X_batch, y_batch in zip(X_batches, y_batches):
            z = softmax(X_batch, w)
            loss = crossEntropy(y_batch, z)
            gradient = gradfCE(X_batch, y_batch, w)
            w -= alpha * gradient
            if printBool and epoch >= 480:
                printBool = False
                print("Epoch", epoch + 1, "training loss:", np.mean(loss))
    return w

def predictClassifications(X, y, w):
    z = np.dot(X, w)
    yhat = np.zeros(np.shape(z))
    encoded_idx = z.argmax(axis = 1)
    yhat[np.arange(np.shape(z)[0]), encoded_idx] = 1
    return yhat

def accuracy(y, yhat):
    y_idx = [np.where(i == 1)[0][0] for i in y]
    yhat_idx = [np.where(i == 1)[0][0] for i in yhat]
    return np.mean(yhat_idx == y_idx)

if __name__ == "__main__":
    # Load Data
    images_test = np.load('small_mnist_test_images.npy')
    images_train = np.load('small_mnist_train_images.npy')
    y_te = np.load('small_mnist_test_labels.npy')
    y_tr = np.load('small_mnist_train_labels.npy')
    X_te = append1s(images_test)
    X_tr = append1s(images_test)
    
    #X_noise = append1s(addRandomNoise(images_train))
    #X_translate = append1s(translateRandom(images_train))
    #X_zoom = append1s(zoomOut(images_train))
    #X_rotate = append1s(rotate15(images_train))
    #X_all = np.concatenate((X_tr, X_noise, X_translate, X_zoom, X_rotate))
    #y_all = np.concatenate((y_tr, y_tr, y_tr, y_tr, y_tr))
    
    w = gradDescent(X_tr, y_tr)
    yhat = predictClassifications(X_te, y_te, w)
    acc = accuracy(y_te, yhat)
    print("Testing Accuracy", acc)
    print("Testing Loss (mean)", np.mean(crossEntropy(y_te, softmax(X_te, w))))
    #showAugmentations(images_train, 0)

Epoch 481 training loss: 0.019883266236342057
Epoch 482 training loss: 0.02046899055591905
Epoch 483 training loss: 0.01749328534548663
Epoch 484 training loss: 0.01927340971231711
Epoch 485 training loss: 0.019724824778073066
Epoch 486 training loss: 0.017591408987556324
Epoch 487 training loss: 0.01963604672293619
Epoch 488 training loss: 0.019159741899054374
Epoch 489 training loss: 0.019811914063746983
Epoch 490 training loss: 0.017892183636987025
Epoch 491 training loss: 0.019226382995180008
Epoch 492 training loss: 0.02038256530538376
Epoch 493 training loss: 0.018991978912120292
Epoch 494 training loss: 0.01883651529287454
Epoch 495 training loss: 0.017945471718375884
Epoch 496 training loss: 0.020391201931325157
Epoch 497 training loss: 0.019076354784223152
Epoch 498 training loss: 0.019011937526815317
Epoch 499 training loss: 0.018814547537283594
Epoch 500 training loss: 0.019672334236013434
Testing Accuracy 0.0
Testing Loss (mean) 0.0005176130061706009


In [3]:
def addRandomNoise(X):
    mask = np.random.randn(X.shape[0], X.shape[1]) * .01
    X_noise = X + mask
    return X_noise

def translateRandom(X):
    m = 5000
    n = 28
    X = X.reshape(m, n, n)
    translations = np.random.randint(3, size = 4) + 1
    for i in range(np.shape(X)[0]):
        # left
        X[i] = np.concatenate((X[i, :, translations[0]:], np.zeros((n, translations[0]))), axis = 1)
        # right
        X[i] = np.concatenate((np.zeros((n, translations[1])), X[i, :, :-translations[1]]), axis = 1)
        # up
        X[i] = np.concatenate((np.zeros((translations[2], n)), X[i, translations[2]:]), axis = 0)
        # down
        X[i] = np.concatenate((X[i, :-translations[3]], np.zeros((translations[3], n))), axis = 0)
    X = X.reshape(5000, n * n)
    return X

def zoomOut(X):
    m = 5000
    n = 28
    X = X.reshape(m, n, n)
    pixelZoom = np.random.randint(2) + 1
    new_n = n + pixelZoom * 2
    for i in range(np.shape(X)[0]):
        temp = X[i]
        temp = np.concatenate((np.zeros((pixelZoom, n)), temp, np.zeros((pixelZoom, n))), axis = 0)
        temp = np.concatenate((np.zeros((new_n, pixelZoom)), temp, np.zeros((new_n, pixelZoom))), axis = 1)
        temp = resize(temp, (n, n))
        X[i] = temp
    X = X.reshape(5000, n * n)
    return X

def rotate15(X):
    m = 5000
    n = 28
    X = X.reshape(m, n, n)
    angle = np.random.randint(15) - 15
    for i in range(np.shape(X)[0]):
        temp = rotate(X[i], angle)
        X[i]= temp
    X = X.reshape(5000, n * n)
    return X

def showAugmentations(X, idx):
    plt.imshow(X[idx].reshape((28, 28)))
    plt.title('original')
    plt.show()
    plt.imshow(addRandomNoise(X)[idx].reshape((28, 28)))
    plt.title('noise')
    plt.show()
    plt.imshow(translateRandom(X)[idx].reshape((28, 28)))
    plt.title('translate')
    plt.show()
    plt.imshow(zoomOut(X)[idx].reshape((28, 28)))
    plt.title('zoom')
    plt.show()
    plt.imshow(rotate15(X)[idx].reshape((28, 28)))
    plt.title('rotate')
    plt.show()

In [42]:
predicted = predictClassifications(X_tr, y_tr, w)
accriate = accuracy(y_tr, predicted)
print(accriate)

0.0
